In [1]:
import os
os.getcwd()

'D:\\Programming Projects\\python projects\\state-of-the-binary-descriptor\\src'

In [2]:
from src import detector_descriptor as dd
from src import data

images = data.load_data('../')
all_detectors = dd.get_all_detectors()
print(all_detectors)

{'AGAST': <class 'cv2.AgastFeatureDetector'>, 'KAZE': <class 'cv2.KAZE'>, 'AKAZE': <class 'cv2.AKAZE'>, 'FAST': <class 'cv2.FastFeatureDetector'>, 'BRISK': <class 'cv2.BRISK'>, 'ORB': <class 'cv2.ORB'>, 'GFTT': <class 'cv2.GFTTDetector'>, 'HarrisLaplace': <class 'cv2.xfeatures2d_HarrisLaplaceFeatureDetector'>, 'StarDetector': <class 'cv2.xfeatures2d_StarDetector'>}
